In [1]:
from getpass import getpass

openai_api_key = getpass("Enter OpenAI Api key: ")

In [2]:
from haystack.components.fetchers.link_content import LinkContentFetcher
from haystack.components.converters import HTMLToDocument
from haystack.components.preprocessors import DocumentSplitter

fetcher = LinkContentFetcher()
converter = HTMLToDocument()
splitter = DocumentSplitter(split_length=100, split_overlap=5)

In [3]:
from haystack.components.rankers import TransformersSimilarityRanker
from haystack.components.generators.openai import OpenAIGenerator

ranker = TransformersSimilarityRanker()
ranker.warm_up()

generator = OpenAIGenerator(api_key=openai_api_key)

In [4]:
from haystack.components.builders.prompt_builder import PromptBuilder

prompt_template = """
According to these documents:

{% for document in documents %}
    {{ document.content }}
{% endfor %}

Answer the given question: {{ question }}
Answer:
"""

prompt_builder = PromptBuilder(template=prompt_template)

In [5]:
from haystack import Pipeline

pipeline = Pipeline()

pipeline.add_component("fetcher", fetcher)
pipeline.add_component("converter", converter)
pipeline.add_component("splitter", splitter)
pipeline.add_component("ranker", ranker)
pipeline.add_component("prompt_builder", prompt_builder)
pipeline.add_component("generator", generator)

pipeline.connect("fetcher.streams", "converter.sources")
pipeline.connect("converter.documents", "splitter.documents")
pipeline.connect("splitter.documents", "ranker.documents")
pipeline.connect("ranker.documents", "prompt_builder.documents")
pipeline.connect("prompt_builder.prompt", "generator.prompt")

In [6]:
question = "What is our favorite animal?"

result = pipeline.run({
    "prompt_builder": {"question": question},
    "ranker": {"query": question},
    "fetcher": {"urls": ["https://haystack.deepset.ai/advent-of-haystack/day-1#challenge"]},
})

print(result["generator"]["replies"][0])

The favorite animal of the Haystack elves is a capybara.


In [7]:
pipeline.draw("pipeline.png")